In [23]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.linear_model import LogisticRegression


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import pandas as pd
import os

# Define the base path
base_path = '/content/drive/MyDrive/Zindi/Digicow/data/'

# Load the datasets
train = pd.read_csv(os.path.join(base_path, 'Train.csv'))
test = pd.read_csv(os.path.join(base_path, 'Test.csv'))
sample_sub = pd.read_csv(os.path.join(base_path, 'SampleSubmission.csv'))

# Verify the data has loaded correctly
print(f"Train set: {train.shape}")
print(f"Test set: {test.shape}")
train.head()

Train set: (5548, 19)
Test set: (2387, 16)


,ID,gender,age,registration,belong_to_cooperative,county,subcounty,ward,trainer,first_training_date,topics_list,num_total_trainings,num_repeat_trainings,days_to_second_training,num_unique_trainers,adopted_within_07_days,adopted_within_90_days,adopted_within_120_days,has_second_training
0,ID_A2UAZE,Female,Below 35,Ussd,0,Bungoma,Bumula,Kabula,trainer_name_2,2024-02-23,"['Ndume App,Poultry Feeding']",1,0,NaN,1,0,0,0,0
1,ID_RM2GTJ,Female,Below 35,Ussd,0,Bungoma,Bumula,Kabula,trainer_name_2,2024-02-24,"['Herd Management,Ndume App,Poultry Feeding,Po...",1,0,NaN,1,0,0,0,0
2,ID_5J3VCK,Male,Above 35,Ussd,0,Nyamira,North Mugirango,Bomwagamo,trainer_name_4,2024-02-21,"['Disadvantages Of Natural Mating', 'Poultry F...",39,38,0.0,1,1,1,1,1
3,ID_2PGL0A,Female,Above 35,Manual,0,Nyamira,North Mugirango,Bomwagamo,trainer_name_4,2024-02-29,"['Factors Affecting Milk Production', 'Factors...",301,300,0.0,1,1,1,1,1
4,ID_K1XGMR,Female,Above 35,Ussd,0,Nyamira,North Mugirango,Bomwagamo,trainer_name_4,2024-02-28,"['Disadvantages Of Natural Mating', 'Poultry F...",19,18,0.0,1,0,0,0,1


In [26]:
## Loading the data
train_df = pd.read_csv(os.path.join(base_path, 'Train.csv'))
test_df = pd.read_csv(os.path.join(base_path, 'Test.csv'))
#y_true = pd.read_csv('reference.csv')['Target_AUC']

train_df_ = pd.read_csv(os.path.join(base_path, 'Train.csv'))
test_df_ = pd.read_csv(os.path.join(base_path, 'Test.csv'))
#y_true_ = pd.read_csv('reference_.csv')['Target_AUC']

In [27]:
for c in [col for col in train_df.columns if col.endswith('date')]:
  if c in train_df.columns:
    train_df[c] = pd.to_datetime(train_df[c], errors='coerce')


for c in [col for col in train_df_.columns if col.endswith('date')]:
  if c in train_df_.columns:
    train_df_[c] = pd.to_datetime(train_df_[c], errors='coerce')


In [28]:
target_col = 'adopted_within_07_days'
train_df[target_col] = train_df[target_col].astype(int)

target_col = 'adopted_within_07_days'
train_df_[target_col] = train_df_[target_col].astype(int)

In [29]:
for c in [col for col in test_df.columns if col.endswith('date')]:
  if c in test_df.columns:
    test_df[c] = pd.to_datetime(test_df[c], errors='coerce')


for c in [col for col in test_df_.columns if col.endswith('date')]:
  if c in test_df_.columns:
    test_df_[c] = pd.to_datetime(test_df_[c], errors='coerce')

In [30]:
split_summary = pd.DataFrame({
    "set": ["train"],
    "rows": [len(train_df)],
    "positives": [train_df[target_col].sum()],
})
split_summary["pos_rate"] = split_summary["positives"] / split_summary["rows"]

split_summary

,set,rows,positives,pos_rate
0,train,5548,615,0.110851


In [31]:
split_summary_ = pd.DataFrame({
    "set": ["train"],
    "rows": [len(train_df_)],
    "positives": [train_df_[target_col].sum()],
})
split_summary_["pos_rate"] = split_summary_["positives"] / split_summary_["rows"]

split_summary_

,set,rows,positives,pos_rate
0,train,5548,615,0.110851


In [32]:
## Basic model with some selected features
base_features = [
    "gender",
    "registration",
    "age",
    "trainer",
    "belong_to_cooperative",
    "county",
    "subcounty",
    "ward",
]

In [33]:
feature_cols = base_features

X_train = train_df[feature_cols]
y_train = train_df[target_col]

X_test  = test_df[feature_cols]


X_train_ = train_df_[feature_cols]
y_train_ = train_df_[target_col]

X_test_  = test_df_[feature_cols]

## Make the prediction


In [34]:
preprocess = ColumnTransformer(
    transformers=[
        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), feature_cols)
    ]
)

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", LogisticRegression(max_iter=3000, class_weight="balanced"))
])

model.fit(X_train, y_train)
y_pred_prob = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

In [35]:
preprocess_ = ColumnTransformer(
    transformers=[
        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ohe", OneHotEncoder(handle_unknown="ignore"))
        ]), feature_cols)
    ]
)

model_ = Pipeline(steps=[
    ("preprocess", preprocess_),
    ("clf", LogisticRegression(max_iter=3000, class_weight="balanced"))
])

model_.fit(X_train_, y_train_)
y_pred_prob_ = model.predict_proba(X_test_)[:, 1]
y_pred_ = model_.predict(X_test_)

In [36]:
y_pred

array([0, 1, 1, ..., 0, 0, 0])

In [37]:
## Evaluating some metrics
# ----------------------------
#  Metrics: PR AUC + ROC AUC + Recall@K
# ----------------------------
def recall_at_k(y_true: pd.Series, y_scores: np.ndarray, k_frac: float) -> float:
    k = int(np.ceil(len(y_true) * k_frac))
    order = np.argsort(-y_scores)  # descending
    topk = order[:k]
    return float(y_true.iloc[topk].sum() / y_true.sum()) if y_true.sum() > 0 else np.nan

In [38]:
# pr_auc = average_precision_score(y_true, y_pred_prob)
# roc_auc = roc_auc_score(y_true, y_pred_prob)
# recall_5 = recall_at_k(y_true, y_pred_prob, 0.05)
# recall_10 = recall_at_k(y_true, y_pred_prob, 0.10)
# recall_20 = recall_at_k(y_true, y_pred_prob, 0.20)

# # Naive baselines on this test set
# prevalence_test = y_true.mean()
# naive_pr_auc = prevalence_test

# results = pd.DataFrame({
#     "metric": [
#         "PR AUC (Average Precision)",
#         "ROC AUC",
#         "Recall@5%",
#         "Recall@10%",
#         "Recall@20%",
#         "Naive PR AUC (prevalence)",
#     ],
#     "value": [pr_auc, roc_auc, recall_5, recall_10, recall_20, naive_pr_auc]
# })


print("\nSplit summary:")
display(split_summary)
# print("\nResults:")
# display(results)


Split summary:


,set,rows,positives,pos_rate
0,train,5548,615,0.110851


In [39]:
y_pred_prob

array([0.01641852, 0.8594947 , 0.89477309, ..., 0.29887217, 0.01665936,
       0.24093631])

In [40]:
y_pred

array([0, 1, 1, ..., 0, 0, 0])

In [41]:
# pr_auc_ = average_precision_score(y_true_, y_pred_prob_)
# roc_auc_ = roc_auc_score(y_true_, y_pred_prob_)
# recall_5_ = recall_at_k(y_true_, y_pred_prob_, 0.05)
# recall_10_ = recall_at_k(y_true_, y_pred_prob_, 0.10)
# recall_20_ = recall_at_k(y_true_, y_pred_prob_, 0.20)

# # Naive baselines on this test set
# prevalence_test_ = y_true_.mean()
# naive_pr_auc_ = prevalence_test_

# results_ = pd.DataFrame({
#     "metric": [
#         "PR AUC (Average Precision)",
#         "ROC AUC",
#         "Recall@5%",
#         "Recall@10%",
#         "Recall@20%",
#         "Naive PR AUC (prevalence)",
#     ],
#     "value": [pr_auc_, roc_auc_, recall_5_, recall_10_, recall_20_, naive_pr_auc_]
# })


print("\nSplit summary:")
display(split_summary_)
# print("\nResults:")
# display(results_)


Split summary:


,set,rows,positives,pos_rate
0,train,5548,615,0.110851


In [43]:
## Make submission file
ss = pd.read_csv(os.path.join(base_path, 'SampleSubmission.csv'))
ss['Target_LogLoss'] = y_pred_prob
ss['Target_AUC'] = y_pred_prob
ss.to_csv('BenchmarkSub.csv', index=False)

ss_ = pd.read_csv(os.path.join(base_path, 'SampleSubmission.csv'))
ss_['Target_LogLoss'] = y_pred_prob_
ss_['Target_AUC'] = y_pred_prob_
ss_.to_csv('BenchmarkSub_.csv', index=False)

In [44]:
ss.head()

,ID,Target_07_AUC,Target_90_AUC,Target_120_AUC,Target_07_LogLoss,Target_90_LogLoss,Target_120_LogLoss,Target_LogLoss,Target_AUC
0,ID_OQZATP,0,0,0,0,0,0,0.016419,0.016419
1,ID_2FAI5D,0,0,0,0,0,0,0.859495,0.859495
2,ID_MYUWBW,0,0,0,0,0,0,0.894773,0.894773
3,ID_EUB4FR,0,0,0,0,0,0,0.608903,0.608903
4,ID_AG7R5S,0,0,0,0,0,0,0.956713,0.956713


In [45]:
ss_.head()

,ID,Target_07_AUC,Target_90_AUC,Target_120_AUC,Target_07_LogLoss,Target_90_LogLoss,Target_120_LogLoss,Target_LogLoss,Target_AUC
0,ID_OQZATP,0,0,0,0,0,0,0.016419,0.016419
1,ID_2FAI5D,0,0,0,0,0,0,0.859495,0.859495
2,ID_MYUWBW,0,0,0,0,0,0,0.894773,0.894773
3,ID_EUB4FR,0,0,0,0,0,0,0.608903,0.608903
4,ID_AG7R5S,0,0,0,0,0,0,0.956713,0.956713


In [46]:
ss

,ID,Target_07_AUC,Target_90_AUC,Target_120_AUC,Target_07_LogLoss,Target_90_LogLoss,Target_120_LogLoss,Target_LogLoss,Target_AUC
0,ID_OQZATP,0,0,0,0,0,0,0.016419,0.016419
1,ID_2FAI5D,0,0,0,0,0,0,0.859495,0.859495
2,ID_MYUWBW,0,0,0,0,0,0,0.894773,0.894773
3,ID_EUB4FR,0,0,0,0,0,0,0.608903,0.608903
4,ID_AG7R5S,0,0,0,0,0,0,0.956713,0.956713
...,...,...,...,...,...,...,...,...,...
2382,ID_MJXKBA,0,0,0,0,0,0,0.030238,0.030238
2383,ID_IT0ZRI,0,0,0,0,0,0,0.010870,0.010870
2384,ID_D7WOHW,0,0,0,0,0,0,0.298872,0.298872
2385,ID_4G191G,0,0,0,0,0,0,0.016659,0.016659


In [47]:
ss_

,ID,Target_07_AUC,Target_90_AUC,Target_120_AUC,Target_07_LogLoss,Target_90_LogLoss,Target_120_LogLoss,Target_LogLoss,Target_AUC
0,ID_OQZATP,0,0,0,0,0,0,0.016419,0.016419
1,ID_2FAI5D,0,0,0,0,0,0,0.859495,0.859495
2,ID_MYUWBW,0,0,0,0,0,0,0.894773,0.894773
3,ID_EUB4FR,0,0,0,0,0,0,0.608903,0.608903
4,ID_AG7R5S,0,0,0,0,0,0,0.956713,0.956713
...,...,...,...,...,...,...,...,...,...
2382,ID_MJXKBA,0,0,0,0,0,0,0.030238,0.030238
2383,ID_IT0ZRI,0,0,0,0,0,0,0.010870,0.010870
2384,ID_D7WOHW,0,0,0,0,0,0,0.298872,0.298872
2385,ID_4G191G,0,0,0,0,0,0,0.016659,0.016659


In [48]:
from sklearn.metrics import log_loss

In [49]:
## Compute the log-loss
# loss = log_loss(y_true, y_pred_prob)
# print("LogLoss:", loss)

# loss_ = log_loss(y_true_, y_pred_prob_)
# print("LogLoss:", loss_)

In [50]:
train_df.columns

Index(['ID', 'gender', 'age', 'registration', 'belong_to_cooperative',
       'county', 'subcounty', 'ward', 'trainer', 'first_training_date',
       'topics_list', 'num_total_trainings', 'num_repeat_trainings',
       'days_to_second_training', 'num_unique_trainers',
       'adopted_within_07_days', 'adopted_within_90_days',
       'adopted_within_120_days', 'has_second_training'],
      dtype='object')

In [51]:
train_df_.columns

Index(['ID', 'gender', 'age', 'registration', 'belong_to_cooperative',
       'county', 'subcounty', 'ward', 'trainer', 'first_training_date',
       'topics_list', 'num_total_trainings', 'num_repeat_trainings',
       'days_to_second_training', 'num_unique_trainers',
       'adopted_within_07_days', 'adopted_within_90_days',
       'adopted_within_120_days', 'has_second_training'],
      dtype='object')